# Daten Bereinigen: Zugriffe auf Webseite

In diesem Beispiel werden wir echte Zugriffe auf eine Webseite analysieren, und so herausfinden, welches die am häufigsten besuchten Unterseiten sind. Klingt einfach, oder?

Zuerst definieren wir einen regulären Ausdruck, der uns bei dem Auslesen einer Zeile aus der Log-Datei unterstützt: 

In [1]:
import re

format_pat= re.compile(
    r"(?P<host>[\d\.]+)\s"
    r"(?P<identity>\S*)\s"
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)


Der Pfad zur Datei, die wir analysieren wollen. Die Datei liegt im selben Ordner, also brauchen wir hier nichts anzupassen.

In [18]:
logPath = "access_log.txt"

Als nächstes schreiben wir ein kleines Script, welches die Datei Zeile für Zeile durchgeht, und die Daten in ein Dictionary schreibt. In diesem Dictionary wird dann als Schlüssel die URL der Seite gepseichert, und als Wert die Anzahl der aufrufe. Was könnte hierbei schief gehen?

In [23]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            request = access['request']
            # request = "GET /blog/ HTTP/1.1" 
            (action, URL, protocol) = request.split()
            if URL in URLCounts:
                URLCounts[URL] = URLCounts[URL] + 1
            else:
                URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

ValueError: not enough values to unpack (expected 3, got 1)

Mhm... Der "request" sollte eigentlich wie folgt aussehen:

`GET /blog/ HTTP/1.1`

Zuerst die HTTP-Methode (GET / POST), dann die URL, und dann das Protokoll. Aber das scheint nicht immer zu klappen. Warum nicht?

In [24]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            request = access['request']
            fields = request.split()
            if (len(fields) != 3):
                print(fields)


['_\\xb0ZP\\x07tR\\xe5']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


Mhm... Also es gibt einige Einträge, in denen der request einfach nur leer ist, bei einem anderen steht dort einfach nur Müll drinnen. Also passen wir unser Script an, dass diese Fälle abgefangen werden:

In [25]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            request = access['request']
            fields = request.split()
            if (len(fields) == 3):
                URL = fields[1]
                if URL in URLCounts:
                    URLCounts[URL] = URLCounts[URL] + 1
                else:
                    URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

/xmlrpc.php: 68494
/wp-login.php: 1923
/: 440
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/world/: 70
/business/: 70
/national-headlines/: 70
/national/: 70
/weather/: 70


Es hat funktioniert! Aber die Ergebnisse machen keinen Sinn. Was wir ja eigentlich wollten, sind die Seiten die von echten Menschen angeschaut werden. Und was ist diese xmlrpc.php? Wenn man sich dazu die Log-Datei näher anschaut, findet man viele Einträge in folgender Form:

`46.166.xxx.xxx - - [05/Dec/2015:05:19:35 +0000] "POST /xmlrpc.php HTTP/1.0" 200 370 "-" "Mozilla/4.0 (compatible: MSIE 7.0; Windows NT 6.0)"`

Was macht dieses Script? Auf jeden Fall wollen wir nur GET - Anfrage betrachten. Warum?

- `GET`: Anzeigen von irgendwelchen Daten im Internet
- `POST`: Verändern / Löschen von irgendwelchen Daten, absenden eines Formulares

In [27]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            request = access['request']
            fields = request.split()
            if (len(fields) == 3):
                (action, URL, protocol) = fields
                if (action == 'GET'):
                    if URL in URLCounts:
                        URLCounts[URL] = URLCounts[URL] + 1
                    else:
                        URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

/: 434
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/world/: 70
/business/: 70
/national-headlines/: 70
/national/: 70
/weather/: 70
/defense-sticking-head-sand/: 69
/about/: 69


Damit haben wir jetzt alle POST - Anfragen aus den Ergebnissen herausgefiltert. Es sieht schonmal besser aus. Aber bei dieser Seite handelt es sich um eine News-Seite - warum lesen so viele Leute den kleinen Blog? Eigentlich sollten sie eher die News-Artikel lesen... 

Wie sieht denn ein typischer Eintrag für /blog/ aus?

54.165.xxx.xxx - - [05/Dec/2015:09:32:05 +0000] "GET /blog/ HTTP/1.0" 200 31670 "-" "-"

Mhm... warum ist bei diesem Eintrag der User-Agent leer? Könnte auf einen bösartigen Scraper oder sonst irgendwas komisches hindeuten. 

Was für User-Agents besuchen eigentlich unserer Webseite?

In [28]:
UserAgents = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            if agent in UserAgents:
                UserAgents[agent] = UserAgents[agent] + 1
            else:
                UserAgents[agent] = 1

results = sorted(UserAgents, key=lambda i: int(UserAgents[i]), reverse=True)

for result in results:
    print(result + ": " + str(UserAgents[result]))

Mozilla/4.0 (compatible: MSIE 7.0; Windows NT 6.0): 68484
-: 4035
Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0): 1724
W3 Total Cache/0.9.4.1: 468
Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html): 278
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html): 248
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36: 158
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0: 144
Mozilla/5.0 (iPad; CPU OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4: 120
Mozilla/5.0 (Linux; Android 5.1.1; SM-G900T Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.76 Mobile Safari/537.36: 47
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm): 43
Mozilla/5.0 (compatible; MJ12bot/v1.4.5; http://www.majestic12.co.uk/bot.php?+): 41
Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.1

Puuhh... Zusätzlich zum User Agent "-" gibt es unglaublich viele verschiedene, automatisierte Scripts die unsere Webseite aufrufen und so unsere Statistik verschmutzen. 

Jetzt könnten wir diese User-Agents manuell herausfiltern, aber das Einfachste ist jetzt erstmal, einfach alle Einträge zu verwerfen, bei denen der User-Agent "-", "bot", "spider" oder "W3 Total Cache" enthält. 

In [17]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            if (not('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent =='-')):
                request = access['request']
                fields = request.split()
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    if (action == 'GET'):
                        if URL in URLCounts:
                            URLCounts[URL] = URLCounts[URL] + 1
                        else:
                            URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

/: 77
/orlando-headlines/: 36
/?page_id=34248: 28
/wp-content/cache/minify/000000/M9AvyUjVzUstLy7PLErVz8lMKkosqtTPKtYvTi7KLCgpBgA.js: 27
/wp-content/cache/minify/000000/lY7dDoIwDIVfiG0KxkfxfnbdKO4HuxICTy-it8Zw15PzfSftzPCckJem-x4qUWArqBPl5mygZLEgyhdOaoxToGyGaiALiOfUnIz0qDLOdSZGE-nOlpc3kopDzrSyavVVt_veb5qSDVhjsQ6dHh_B_eE_z2pYIGJ7iBWKeEio_eT9UQe4xHhDll27mGRryVu_pRc.js: 27
/wp-content/cache/minify/000000/M9bPKixNLarUy00szs8D0Zl5AA.js: 27
/wp-content/cache/minify/000000/fY45DoAwDAQ_FMvkRQgFA5ZyWLajiN9zNHR0O83MRkyt-pIctqYFJPedKyYzfHg2PzOFiENAzaD07AxcpKmTolORvDjZt8KEfhBUGjZYCf8Fb0fvA1TXCw.css: 25
/?author=1: 21
/wp-content/cache/minify/000000/hcrRCYAwDAXAhXyEjiQ1YKAh4SVSx3cE7_uG7ASr4M9qg3kGWyk1adklK84LHtRj_My6Y0Pfqcz-AA.js: 20
/wp-content/uploads/2014/11/nhn1.png: 19
/wp-content/cache/minify/000000/BcGBCQAgCATAiUSaKYSERPk3avzuht4SkBJnt4tHJdqgnPBqKldesTcN1R8.js: 17
/wp-includes/js/wp-emoji-release.min.js?ver=4.3.1: 17
/wp-login.php: 16
/comics-2/: 12
/world/: 12
/favicon.ico: 10
/wp-content/up

Jetzt haben wir ein neues Problem: Wir sehen, dass viele Dateien angefragt werden, Dateien die überhaupt keine Webseiten sind, sondern nur von .html - Dateien eingebunden werden. Diese müssen wir natürlich ignorieren, wir interessieren uns dafür ja nicht.

Daher entfernen wir im nächsten Schritt alle Adresse, die nicht auf den Schrägstrich "/" enden. Das liegt daran, dass ich weiß, dass all meine Artikel eine URL haben, die auf einen Schrägstrich endet - daher darf ich das tun. Hier verwende ich also weiteres Wissen, welches ich über die Daten habe, um diesen Schritt zu begründen.

In [29]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match= format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            if (not('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent =='-')):
                request = access['request']
                fields = request.split()
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    if (URL.endswith("/")):
                        if (action == 'GET'):
                            if URL in URLCounts:
                                URLCounts[URL] = URLCounts[URL] + 1
                            else:
                                URLCounts[URL] = 1

results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/about/: 4
/weather/: 4
/australia/: 4
/national-headlines/: 3
/science/: 2
/feed/: 2
/sample-page/feed/: 2
/technology/: 2
/entertainment/: 1
/travel/feed/: 1
/san-jose-headlines/: 1
/business/: 1


Das sieht jetzt schonmal etwas glaubwürdiger aus. Aber wenn man noch etwas weiter in die Daten hineinschaut sieht man, dass die Seiten mit "/feed/" etwas komisch aussehen, und dass immernoch ein paar Zugriffe von Suchmaschienen nicht herausgefiltert wurden. Dennoch - auf Basis dieser Daten sieht es so aus, dass Orlando News, World News und Comics die am häufigsten aufgerufenen Seiten sind. 

Das Fazit: Kenn dich mit den Daten aus. Hinterfrag immer was die Ergebnisse sind, und überprüfe die Ergebnisse, bevor du vorschnelle Schlüsse ziehst. Wenn deine Firma wegen einer leichtfertigen Entscheidung nachher richtig Geld verliert, kann das definitiv zu Problemen führen.

Zudem: Bei jeden Schritt, überlege dir gut, ob du ihn durchführen darfst. Warum darfst du die Daten in jedem Schritt bereinigen? Bereinige die Daten nicht, weil dir das Ergebnis am Ende nicht passt!

## Aufgabe

Dieser Ergebnisse sind noch nicht perfekt, die URL enthalten teilweise noch das Wort "feed", diese werden auch automatisiert abgerufen. Passe den Code so an, dass auch diese Einträge ignoriert werden. Wenn du Lust hast, schau dir das Log-File noch etwas genauer an - was für User-Agents rufen die /feed - Seiten auf? Wo könnten diese herkommen? 